In [235]:
#!/usr/bin/env python

"""
takes all rows for the 50 most-posted-in groups, removes the weird users,
and outputs the subset along with unique user ids.
Takes the EP_data.csv file.
"""

import pandas as pd
import numpy as np

#read in the full Experience Project file
datafile = pd.read_csv('EP_data.csv')

#get a list of the 50 groups with the most posts
group_count = pd.Series(datafile["gid"])
top50 = group_count.value_counts()[0:50]

In [240]:
#and if you'd like to see the groups along with more info:
#print (top50)
#sumtop50 = top50.sum()
#print ('there are %d total posts in the top 50 groups') % sumtop50

In [241]:
groups = []
for index, row in top50.iteritems():
    groups.append(index)
#print 'The top 50 groups are:',groups

In [242]:
#make a smaller dataset containing only entries with top group ids
subset_dirty = datafile[datafile['gid'].isin(groups)]
#print len(subset_dirty), 'should equal the number of posts calculated above:', sumtop50
#clean subset, removing entries by suspected moderator
subset = subset_dirty[subset_dirty['num_entries']!=65535]
#print 'after removing fake user, the list is this long:',len(subset)

In [243]:
#add user id to the subset
#read in file with member ids matched to other data
members = pd.read_csv('members.csv')
#print members.columns
#print subset.columns

In [246]:
#merge our subset with member based upon matching data points
IDsubset = pd.merge(subset, members, how='inner', on=['created','birthdate','gender'])

#details of each dataframe shows how many entries are in each
#print IDsubset.head(n = 1)
#print subset.shape
#print members.shape
#print IDsubset.shape

In [247]:
#I rename the dataframe to do some acrobatics with it
df = IDsubset
#make a new column that we will use to remove duplicates from our merge step
df['gcbp'] = df.gender+df.created+df.birthdate+df.date_posted
#sort dataframe so that duplicates are ordered by time of last login
sortedIDsubset = df.sort_index(by=['gcbp','last_login_y'])
#sortedIDsubset.head()
#remove all duplicates from the 'gcbp' column, leaving unique entries
#with a member id
df2 = sortedIDsubset.drop_duplicates(subset ='gcbp')

#give dataframe its old name and remove/rename unnecessary columns
IDsubset = df2.drop(['gcbp','last_login_y'], 1)
IDsubset = IDsubset.rename(columns = {'last_login_x':'last_login'})

In [248]:
IDsubset.to_csv('topgroups')